In [126]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import time
from tqdm import tqdm
import string

In [102]:
alphabet = list(string.ascii_uppercase)
alphabet.append('Æ')
alphabet.append('Ø')
alphabet.append('Å')

page=list(range(4))

In [107]:
url_links_page = []
for i in range(len(alphabet)):
    url_links = 'https://www.apoteket.dk/alle-apoteker?letter='+alphabet[i]
    for u in range(len(page)):
        url_links_page.append(url_links+'&page='+str(page[u]))

In [121]:
#for links in url_links_page:
links = []
for link in range(len(url_links_page)):
    url = url_links_page[link]
    response = requests.get(url)
    link_locations = response.text.split('href="/apoteker/')
    for link in link_locations:
        link = link.split('"')[0]    
        links.append(link)
    time.sleep(0.5)

In [134]:
print(len(links))
final_links = list(set(links))
print(len(final_links))

1304
1188


In [142]:
final_links_test = final_links[0:20]
final_links_test

['gentofte-apotek',
 'nykoebing-f-svane-apotek/guldborgsund-svane-apotek',
 'holbaek-elefant-apotek/oroe-haandkoebsudsalg',
 'holstebro-svane-apotek/vinderup-apotek',
 'storvorde-sejlflod-apotek/baelum-medicinudleveringssted',
 'naestved-loeve-apotek/mogenstrup-blomster-bek-medicinudleveringssted',
 'soenderborg-loeve-apotek-center-oest/fynshav-medicinudleveringssted',
 'uldum-apotek/klovborg-medicinudleveringssted',
 'asnaes-apotek',
 'esbjerg-jerne-apotek/hjerting-kiosken',
 'helsinge-apotek/blistrup-haandkoebsudsalg',
 'soenderborg-jernbane-apotek/apoteket-borgen',
 'birkeroed-apotek',
 'odense-bellinge-apotek/apoteket-dalumcentret',
 'viborg-loeve-apotek/houlkaer-haandkoebsudsalg',
 'skoerping-apotek',
 'ringkoebing-loeve-apotek/hvide-sande-apoteksudsalg',
 'slagelse-svane-apotek',
 'nexoe-apotek/allinge-apotek/tejn-haandkoebsudsalg',
 'koebenhavn-sydhavnsapoteket']

In [146]:
link_p1 = 'https://www.apoteket.dk/apoteker/'

dataframes = []
for link in tqdm(final_links):
    response = requests.get(link_p1+link)
    soup = BeautifulSoup(response.content, "html.parser")
    table = soup.find("table") # Find the "table" tag in the page
    try:
        rows = table.find_all("tr") # Find all the "tr" tags in the table 
        header = soup.find('h3', class_="table__headline").text.strip()[11:]
    except AttributeError:
        continue
    cy_data = []
    for row in rows:
        cells = row.find_all("td") #  Find all the "td" tags in each row
        cells = cells#[1:] # Select the correct columns
        cy_data.append([cell.text.strip('\n') for cell in cells]) # For each "td" tag, get the text inside it
    for i in range(len(cy_data)):
        cy_data[i].insert(0, header) 

    dataframes.append(pd.DataFrame(cy_data).drop(0, axis=0))
    time.sleep(0.5)

data = pd.concat(dataframes)

100%|██████████████████████████████████████████████████████████████████████████████| 1188/1188 [12:15<00:00,  1.61it/s]


In [147]:
data_name = data.rename(columns = {0: 'Apotek', 1: 'Tidspunkt',2: 'mandad', 3: 'tirsdag',4: 'onsdag',5: 'torsdag',6: 'fredag',7: 'lørdag'}, inplace = False)
#data_melt = data.melt(id_vars=['Apotek'].sort_values(by='Apotek').reset_index(drop=True))
#data_melt_value = data_melt(id_vars='value')
#data_melt_value
#data_melt = data.melt(0, var_name = 'apotek', value_name = 'ventetid').sort_values(by='apotek').reset_index(drop=True)

In [148]:
data_name

,Apotek,Tidspunkt,mandad,tirsdag,onsdag,torsdag,fredag,lørdag
1,Gentofte Apotek,Morgen 9-11,2:22,1:49,1:50,2:27,3:17,1:27
2,Gentofte Apotek,Frokost 11-13,2:45,2:24,2:10,3:08,3:32,2:38
3,Gentofte Apotek,Tidlig eftermiddag 13-15,3:00,2:31,1:59,2:36,2:51,1:43
4,Gentofte Apotek,Sen eftermiddag efter 15,3:03,2:28,1:46,2:00,2:50,
5,Gentofte Apotek,Hele dagen,2:50,2:21,1:56,2:31,3:07,2:06
1,Guldborgsund Svane Apotek,Morgen 9-11,1:17,1:27,1:19,1:42,1:42,2:03
2,Guldborgsund Svane Apotek,Frokost 11-13,2:02,1:36,2:17,1:45,2:06,2:43
3,Guldborgsund Svane Apotek,Tidlig eftermiddag 13-15,0:58,0:59,0:49,1:57,1:13,1:16
4,Guldborgsund Svane Apotek,Sen eftermiddag efter 15,1:35,2:37,1:35,1:46,1:58,
5,Guldborgsund Svane Apotek,Hele dagen,1:32,1:45,1:30,1:48,1:47,2:11


In [149]:
data_name.to_excel("Resultat_scrap_17_9.xlsx") 

In [151]:
check = len(set(data_name['Apotek']))